In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%%writefile model.py
import torch
import torchvision
import torchvision.transforms as transforms
import random
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# -------------------------
# h) Reproducibility (Requirement g)
# -------------------------
def set_seed(seed=42):
    # Set random seeds for reproducibility 
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # CuDNN deterministic flags (Requirement g)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()
#Data loading and pre-processing
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

#Model architecture
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

#defining a loss function and optimiser
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

#Training loop implementation
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

#saving the trained model
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

#Evaluation and metrics
dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

net = Net()
net.load_state_dict(torch.load(PATH, weights_only=True))

outputs = net(images)
outputs

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

net.to(device)

inputs, labels = data[0].to(device), data[1].to(device)


Writing model.py


In [3]:
%%writefile train_ddp.py
from model import Net
import torch
import os
import torchvision
import torchvision.transforms as transforms
import random
import numpy as np
import torch.nn.functional as F
import torch.multiprocessing as mp
from torch.utils.data import DataLoader, Dataset 
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

def ddp_setup(rank: int, world_size: int):
   """
   Args:
       rank: Unique identifier of each process
      world_size: Total number of processes
   """
   os.environ["MASTER_ADDR"] = "localhost"
   os.environ["MASTER_PORT"] = "12355"
   torch.cuda.set_device(rank)  # sets the default GPU for each process
  #Using NCCL(NVIDIA Collective Communications Library (NCCL)) backend for distributed training with CUDA GPU
   init_process_group(backend="nccl", rank=rank, world_size=world_size) 

#model = net

class Trainer:
    def __init__(
        self,
        model: torch.nn.Module,
        train_data: DataLoader,
        optimizer: torch.optim.Optimizer,
        gpu_id: int,
        save_every: int,
    ) -> None:
        self.gpu_id = gpu_id
        self.model = model.to(gpu_id)
        self.train_data = train_data
        self.optimizer = optimizer
        self.save_every = save_every
        self.model = DDP(model, device_ids=[gpu_id])

    def _run_batch(self, source, targets):
        self.optimizer.zero_grad()
        output = self.model(source)
        loss = F.cross_entropy(output, targets)
        loss.backward()
        self.optimizer.step()

    def _run_epoch(self, epoch):
        b_sz = len(next(iter(self.train_data))[0])
        print(f"[GPU{self.gpu_id}] Epoch {epoch} | Batchsize: {b_sz} | Steps: {len(self.train_data)}")
        self.train_data.sampler.set_epoch(epoch)
        for source, targets in self.train_data:
            source = source.to(self.gpu_id)
            targets = targets.to(self.gpu_id)
            self._run_batch(source, targets)

    def _save_checkpoint(self, epoch):
        ckp = self.model.module.state_dict()
        PATH = "checkpoint.pt"
        torch.save(ckp, PATH)
        print(f"Epoch {epoch} | Training checkpoint saved at {PATH}")

    def train(self, max_epochs: int):
        for epoch in range(max_epochs):
            self._run_epoch(epoch)
            if self.gpu_id == 0 and epoch % self.save_every == 0:
                self._save_checkpoint(epoch)

def load_train_objs():
    # 1️⃣ Load CIFAR-10 training dataset
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

    # 2️⃣ Initialize your model (replace Net() with your actual model class)
    net = Net()
    net.load_state_dict(torch.load('./cifar_net.pth'))  # load trained weights
    model = net

    # 3️⃣ Create optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    return train_set, model, optimizer

def prepare_dataloader(dataset: Dataset, batch_size: int, rank: int, world_size: int):
    sampler = DistributedSampler(
        dataset,
        num_replicas=world_size,
        rank=rank,
        shuffle=True  # shuffling done here per epoch
    )
    
    return DataLoader(
        dataset,
        batch_size=batch_size,
        pin_memory=True,
        sampler=sampler
    )

def main(rank: int, world_size: int, save_every: int, total_epochs: int, batch_size: int):
    ddp_setup(rank, world_size)
    dataset, model, optimizer = load_train_objs()
    train_data = prepare_dataloader(dataset, batch_size, rank, world_size)
    trainer = Trainer(model, train_data, optimizer, rank, save_every)
    trainer.train(total_epochs)
    destroy_process_group()

if __name__ == "__main__":
    # Training settings (set manually for notebook)
    total_epochs = 10     # total number of epochs
    save_every = 2        # how often to save checkpoints
    batch_size = 32       # batch size per GPU

    # Number of GPUs available
    world_size = torch.cuda.device_count()
    if world_size == 0:
        raise RuntimeError("No CUDA devices available for DDP.")

    # Launch distributed training
    mp.spawn(
        main,
        args=(world_size, total_epochs, save_every, batch_size),
        nprocs=world_size
    )

Writing train_ddp.py


In [4]:
%%writefile train.sh
#! /bin/bash

echo Starting training script

python train_ddp.py

echo finished training successfully

Writing train.sh


In [5]:
!cat ./train.sh

#! /bin/bash

echo Starting training script

python train_ddp.py

echo finished training successfully


In [6]:
!chmod +x ./train.sh

In [7]:
ls

model.py  __notebook__.ipynb  train_ddp.py  train.sh*


In [8]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /kaggle/working/.git/


In [9]:
!git add .

In [10]:
!git status

On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   __notebook__.ipynb
	new file:   model.py
	new file:   train.sh
	new file:   train_ddp.py

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   __notebook__.ipynb



In [11]:
%%bash
git config  user.email "20148527@tafe.wa.edu.au"
git config  user.name "sulak2025"

In [12]:
!git commit -m "initial commit"

[master (root-commit) 4ae98ea] initial commit
 4 files changed, 2100 insertions(+)
 create mode 100644 __notebook__.ipynb
 create mode 100644 model.py
 create mode 100755 train.sh
 create mode 100644 train_ddp.py


In [13]:
!git remote add origin https://github.com/sulak2025/at2-pytorch.git

In [14]:
!git branch

* master


In [15]:
!git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   __notebook__.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [16]:
!git branch -m master main

In [17]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
github_token = user_secrets.get_secret("GitHub")

In [18]:
!cat .git/config

[core]
	repositoryformatversion = 0
	filemode = true
	bare = false
	logallrefupdates = true
[user]
	email = 20148527@tafe.wa.edu.au
	name = sulak2025
[remote "origin"]
	url = https://github.com/sulak2025/at2-pytorch.git
	fetch = +refs/heads/*:refs/remotes/origin/*


In [19]:
!ls -l .git

total 44
drwxr-xr-x  2 root root 4096 Nov 18 08:08 branches
-rw-r--r--  1 root root   15 Nov 18 08:08 COMMIT_EDITMSG
-rw-r--r--  1 root root  265 Nov 18 08:08 config
-rw-r--r--  1 root root   73 Nov 18 08:08 description
-rw-r--r--  1 root root   21 Nov 18 08:08 HEAD
drwxr-xr-x  2 root root 4096 Nov 18 08:08 hooks
-rw-r--r--  1 root root  377 Nov 18 08:08 index
drwxr-xr-x  2 root root 4096 Nov 18 08:08 info
drwxr-xr-x  3 root root 4096 Nov 18 08:08 logs
drwxr-xr-x 10 root root 4096 Nov 18 08:08 objects
drwxr-xr-x  4 root root 4096 Nov 18 08:08 refs


In [20]:
%%writefile .git/config
[core]
	repositoryformatversion = 0
	filemode = true
	bare = false
	logallrefupdates = true
[remote "origin"]
	url = https://github.com/sulak2025/at2-pytorch.git
	fetch = +refs/heads/*:refs/remotes/origin/*
[user]
	email = 20148527@tafe.wa.edu.au
	name = sulak2025

Overwriting .git/config


In [21]:
!cat .git/config

[core]
	repositoryformatversion = 0
	filemode = true
	bare = false
	logallrefupdates = true
[remote "origin"]
	url = https://github.com/sulak2025/at2-pytorch.git
	fetch = +refs/heads/*:refs/remotes/origin/*
[user]
	email = 20148527@tafe.wa.edu.au
	name = sulak2025


In [22]:
!gh auth status

You are not logged into any GitHub hosts. Run gh auth login to authenticate.


In [23]:
!gh auth setup-git

You are not logged into any GitHub hosts. Run gh auth login to authenticate.


In [24]:
! export GITHUB_TOKEN={github_token}

In [25]:
! echo {github_token} | gh auth login --with-token 

error validating token: missing required scope 'read:org'


In [26]:
!git pull

remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 13 (delta 3), reused 10 (delta 3), pack-reused 3 (from 1)
Unpacking objects: 100% (13/13), 22.79 KiB | 3.80 MiB/s, done.
From https://github.com/sulak2025/at2-pytorch
 * [new branch]      main       -> origin/main
There is no tracking information for the current branch.
Please specify which branch you want to merge with.
See git-pull(1) for details.

    git pull <remote> <branch>

If you wish to set tracking information for this branch you can do so with:

    git branch --set-upstream-to=origin/<branch> main



In [27]:
! git remote set-url origin https://{github_token}@github.com/sulak2025/at2-pytorch.git

In [28]:
!git lfs install

Updated git hooks.
Git LFS initialized.


In [29]:
!git lfs track *.gz

Tracking "*.gz"


In [ ]:
!git add .

In [ ]:
!git commit -m "enable lfs gz" 

In [ ]:
!git status

In [ ]:
!git push -u origin main

In [ ]:
!git push origin main --force

In [ ]:
!git pull

In [ ]:
!git push -u origin main

In [ ]:
!cat /root/.gitconfig

In [ ]:
!./train.sh

In [ ]:
!python train_ddp.py

In [ ]:
!pip freeze > requirements.txt

In [ ]:
!cat requirements.txt

In [ ]:
!git pull origin main

In [ ]:
!git add requirements.txt

In [ ]:
!git commit -m "add requirements.txt"

In [ ]:
!git push -u origin main

In [ ]:
! ls -la

In [ ]:
%%writefile .gitignore
__pycache__/
*.pyc
*.virtual_documents

In [ ]:
!git rm -r --cached __pycache__

In [ ]:
!git add .gitignore
!git commit -m "Remove unwanted files and add .gitignore"

In [ ]:
!git push -u origin main